# **JohnSnowlabsLabs LLM Playground**

### Imports and Constants

In [1]:
from pprint import pprint as print
import requests
import json
import time
from IPython.display import clear_output


APP_URL = "http://jsl-llms-api:5000"
HEADERS = {
  'Content-Type': 'application/json'
}

## Status check
Please wait until you see the status response

In [2]:
requests.get(f"{APP_URL}/status").json()

{'deployed': {'model': 'jsl_meds_ner_q4_v2',
  'device': 'auto',
  'max_context_length': 4096,
  'deployed_at': '2024-09-16 07:43:22 UTC'},
 'system': {'torch_cpu_threads': 2,
  'is_cuda_available': True,
  'number_of_cuda_devices': 1,
  'gpus_info': [{'GPU ID': 0,
    'GPU Name': 'NVIDIA L40S',
    'GPU Load': '0.0 %',
    'GPU Memory Free': '42717.0 MB',
    'GPU Memory Used': '2878.0 MB',
    'GPU Memory Total': '46068.0  MB',
    'GPU Temperature': '45.0 °C',
    'GPU Driver': '535.183.01'}]}}

## Available JSL llms

In [3]:
requests.get(f"{APP_URL}/info").json()

{'jsl_medm_q16_v2': {'size': '14.97 GB'},
 'jsl_medm_q4_v2': {'size': '5.62 GB'},
 'jsl_medm_q8_v2': {'size': '8.47 GB'},
 'jsl_meds_q8_v1': {'size': '3.75 GB'},
 'jsl_meds_q16_v1': {'size': '21.35 GB'},
 'jsl_meds_q16_v2': {'size': '21.35 GB'},
 'jsl_meds_q4_v1': {'size': '2.27 GB'},
 'jsl_meds_ner_q16_v1': {'size': '7.12 GB'},
 'jsl_meds_ner_q16_v2': {'size': '7.12 GB'},
 'jsl_meds_ner_q4_v2': {'size': '2.27 GB'},
 'jsl_meds_ner_q8_v2': {'size': '3.75 GB'},
 'jsl_meds_rag_q8_v1': {'size': '3.75 GB'},
 'jsl_meds_rag_q4_v1': {'size': '2.27 GB'},
 'jsl_meds_rag_q16_v1': {'size': '7.12 GB'}}

## Deploy a new llm

In [4]:
payload = json.dumps({
  "llm_name": "jsl_meds_ner_q4_v2",
  "force": True # removes any cache
})

requests.post(f"{APP_URL}/deploy", headers=HEADERS, data=payload).json()

{'message': 'deployment started'}

## Track the deployment status

In [5]:
def track_deployment_status(poll_every_seconds=10):
    for i in range(50):
        status_resp = requests.get(f"{APP_URL}/status").json()
        clear_output(wait=True)  # Clears the previous output
        print(status_resp)
        # deployment_status is not available after deployment is complete
        if not (deployment_status := status_resp.get("deployment_status")) or deployment_status.get("status") == "failed":
            return
        time.sleep(poll_every_seconds)

track_deployment_status()

{'deployed': {'deployed_at': '2024-09-16 07:46:17 UTC',
              'device': 'auto',
              'max_context_length': 4096,
              'model': 'jsl_meds_ner_q4_v2'},
 'system': {'gpus_info': [{'GPU Driver': '535.183.01',
                           'GPU ID': 0,
                           'GPU Load': '0.0 %',
                           'GPU Memory Free': '42717.0 MB',
                           'GPU Memory Total': '46068.0  MB',
                           'GPU Memory Used': '2878.0 MB',
                           'GPU Name': 'NVIDIA L40S',
                           'GPU Temperature': '43.0 °C'}],
            'is_cuda_available': True,
            'number_of_cuda_devices': 1,
            'torch_cpu_threads': 2}}


## Inference

In [6]:
payload = json.dumps({
  "text": "### Template:\n{\n\"Drugs\": []\n}\n### Text:\nShe is consuming lipitor and metformin daily"
})
requests.post(f"{APP_URL}/generate", headers=HEADERS, data=payload).json()

{'message': '{\n"Drugs": [\n    "lipitor",\n    "metformin"\n]\n}\n',
 'generation_started': '2024-09-16 07:46:23 UTC',
 'time_taken': '1064.000 ms',
 'new_token_generated': 27,
 'input': {'max_new_tokens': 128, 'temperature': 0.1, 'prompt_tokens': 31},
 'deployment': {'deployed_at': '2024-09-16 07:46:17 UTC',
  'model_name': 'jsl_meds_ner_q4_v2',
  'device': 'auto'}}

## Generic inference function

In [7]:
def generate(prompt, payload=payload,  max_new_tokens=128):
    url = f"{APP_URL}/generate"

    payload = json.dumps({
      "text": prompt,
       "max_new_tokens" : max_new_tokens
    })
    response = requests.request("POST", url, headers=HEADERS, data=payload)
    return response.json()

# JSL LLM Types

## JSL_MedS_NER

* 'jsl_meds_ner_q16_v1',
* 'jsl_meds_ner_q16_v2',
* 'jsl_meds_ner_q4_v2',
* 'jsl_meds_ner_q8_v2',

In [8]:
payload = json.dumps({
  "llm_name": "jsl_meds_ner_q16_v2"
})
requests.request("POST", f"{APP_URL}/deploy", headers=HEADERS, data=payload).json()

{'message': 'deployment started'}

In [9]:
# Track deployment status
track_deployment_status(poll_every_seconds=20)

{'deployed': {'deployed_at': '2024-09-16 07:52:30 UTC',
              'device': 'auto',
              'max_context_length': 4096,
              'model': 'jsl_meds_ner_q16_v2'},
 'system': {'gpus_info': [{'GPU Driver': '535.183.01',
                           'GPU ID': 0,
                           'GPU Load': '0.0 %',
                           'GPU Memory Free': '37769.0 MB',
                           'GPU Memory Total': '46068.0  MB',
                           'GPU Memory Used': '7826.0 MB',
                           'GPU Name': 'NVIDIA L40S',
                           'GPU Temperature': '38.0 °C'}],
            'is_cuda_available': True,
            'number_of_cuda_devices': 1,
            'torch_cpu_threads': 2}}


### q0

In [10]:
text = "She is consuming lipitor and metformin daily"

template = """
{
    "Drugs": []
}
"""

prompt = f"""<|input|>
### Template:
{template}
### Text:
{text}
<|output|>
"""

response = generate(prompt = prompt)
eval(response["message"])

{'Drugs': ['lipitor', 'metformin']}

### q1

In [11]:
text = """
I feel a bit drowsy & have a little blurred vision , and some gastric problems .
I 've been on Arthrotec 50 for over 10 years on and off , only taking it when I needed it .
"""

template = """
{
    "drugs": [
        {
            "name": "",
            "reactions": []
        }
    ]
}
"""

prompt = f"""<|input|>
### Template:
{template}
### Text:
{text}
<|output|>
"""

response = generate(prompt = prompt)
eval(response["message"])

{'drugs': [{'name': 'Arthrotec', 'reactions': ['drowsy']}]}

### q2

In [12]:
text = """
David Hale, M.D. he is 45 years old and works as Data Engineer at GE Healthcare, which is located in Delaware. For personal Contact and Email: 555-555-555, hale@gmail.com,
Dr. Molina Cortes, MD. is affiliated with the Cocke County Baptist Hospital in Des Moines.
"""

import json
ents = {
  "DATE": [],
  "NAME": [],
  "AGE": [],
  "LOCATION": [],
  "PROFESSION": [],
  "CONTACT": [],
  "Email": []
}

template = f"""{json.dumps(ents, indent=2)}"""

prompt = f"""<|input|>
### Template:
{template}
### Text:
{text}
<|output|>"""

response = generate(prompt = prompt, max_new_tokens = 256)
print(response["message"])

('{\n'
 '  "DATE": [],\n'
 '  "NAME": [\n'
 '    "David Hale",\n'
 '    "Molina Cortes",\n'
 '    "hale"\n'
 '  ],\n'
 '  "AGE": [\n'
 '    "45"\n'
 '  ],\n'
 '  "LOCATION": [\n'
 '    "Delaware",\n'
 '    "Cocke County",\n'
 '    "Des Moines"\n'
 '  ],\n'
 '  "PROFESSION": [\n'
 '    "Data Engineer"\n'
 '  ],\n'
 '  "CONTACT": [\n'
 '    "555-555-555"\n'
 '  ],\n'
 '  "Email": [\n'
 '    "hale@gmail.com"\n'
 '  ]\n'
 '}\n')


### q3

In [13]:
text = """
I feel a bit drowsy & have a little blurred vision , and some gastric problems .
I 've been on Arthrotec 50 for over 10 years on and off , only taking it when I needed it .
Due to my arthritis getting progressively worse , to the point where I am in tears with the agony.
Gp 's started me on 75 twice a day and I have to take it every day for the next month to see how I get on , here goes .
So far its been very good , pains almost gone , but I feel a bit weird , did n't have that when on 50.
"""

import json
ents = {
  "drugs": [],
  "adverse drug event": []
}

ner_prompt = f"""<|input|>
### Template:
{json.dumps(ents, indent=2)}

### Text:
{text}
<|output|>
"""

response = generate(prompt = ner_prompt, max_new_tokens = 256)
print(response["message"])

('{\n'
 '  "drugs": [\n'
 '    "Arthrotec",\n'
 '    "Arthrotec 50",\n'
 '    "75"\n'
 '  ],\n'
 '  "adverse drug event": [\n'
 '    "drowsy",\n'
 '    "blurred vision",\n'
 '    "gastric problems"\n'
 '  ]\n'
 '}\n')


### q4

In [14]:
text = """
A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total anterior hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago.
Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2).
Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months.
Core needle biopsy revealed metaplastic carcinoma. Neoadjuvant chemotherapy with the regimens of Taxotere (75 mg/m2), Epirubicin (75 mg/m2 Dosage), and Cyclophosphamide (500 mg/m2) was given for 6 cycles with poor response, followed by a modified radical mastectomy (MRM) with dissection of axillary lymph nodes and skin grafting.
Postoperatively, radiotherapy was done with 5000 cGy in 25 fractions. The histopathologic examination revealed a metaplastic carcinoma with squamous differentiation associated with adenomyoepithelioma.
Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin (AE1/AE3) stain, and myoepithelial markers, including cytokeratin 5/6 (CK 5/6), p63, and S100 stains.
The dissected axillary lymph nodes showed metastastic carcinoma with negative hormone receptors in 3 nodes. The patient was staged as pT3N1aM0, with histologic tumor grade III
"""

import json
ents = {
  "Biomarker": [],
  "Cancer_Dx": [],
  "Cancer_Surgery": [],
  "Chemotherapy": [],
  "Grade": [],
  "Metastasis": [],
  "Tumor_Size": [],
  "Unspecific_Therapy": []
}

ner_prompt = f"""<|input|>
### Template:
{json.dumps(ents, indent=2)}

### Text:
{text}
<|output|>
"""

response = generate(prompt = ner_prompt,max_new_tokens = 512)
print(response["message"])

('{\n'
 '  "Biomarker": [\n'
 '    "AE1/AE3",\n'
 '    "CK 5/6",\n'
 '    "p63",\n'
 '    "S100"\n'
 '  ],\n'
 '  "Cancer_Dx": [\n'
 '    "ovarian carcinoma",\n'
 '    "mucinous-type carcinoma",\n'
 '    "metaplastic carcinoma",\n'
 '    "adenomyoepithelioma"\n'
 '  ],\n'
 '  "Cancer_Surgery": [\n'
 '    "debulking surgery",\n'
 '    "bilateral oophorectomy",\n'
 '    "total anterior hysterectomy",\n'
 '    "radical pelvic lymph nodes dissection",\n'
 '    "neoadjuvant chemotherapy",\n'
 '    "modified radical mastectomy",\n'
 '    "radiotherapy"\n'
 '  ],\n'
 '  "Chemotherapy": [\n'
 '    "cyclophosphamide",\n'
 '    "carboplatin",\n'
 '    "Taxotere",\n'
 '    "Epirubicin",\n'
 '    "Cyclophosphamide"\n'
 '  ],\n'
 '  "Grade": [\n'
 '    "III"\n'
 '  ],\n'
 '  "Metastasis": [\n'
 '    "metastastic carcinoma"\n'
 '  ],\n'
 '  "Tumor_Size": [\n'
 '    "15 cm"\n'
 '  ],\n'
 '  "Unspecific_Therapy": []\n'
 '}\n'
 '\n')


## JSL_MedM
* 'jsl_medm_q16_v2',
* 'jsl_medm_q4_v2',
* 'jsl_medm_q8_v2',

In [15]:
payload = json.dumps({
  "llm_name": "jsl_medm_q16_v2"
})
requests.request("POST", f"{APP_URL}/deploy", headers=HEADERS, data=payload).json()

{'message': 'deployment started'}

In [16]:
track_deployment_status(poll_every_seconds=30)

{'deployed': {'deployed_at': '2024-09-16 08:06:55 UTC',
              'device': 'auto',
              'max_context_length': 131072,
              'model': 'jsl_medm_q16_v2'},
 'system': {'gpus_info': [{'GPU Driver': '535.183.01',
                           'GPU ID': 0,
                           'GPU Load': '0.0 %',
                           'GPU Memory Free': '29613.0 MB',
                           'GPU Memory Total': '46068.0  MB',
                           'GPU Memory Used': '15982.0 MB',
                           'GPU Name': 'NVIDIA L40S',
                           'GPU Temperature': '40.0 °C'}],
            'is_cuda_available': True,
            'number_of_cuda_devices': 1,
            'torch_cpu_threads': 2}}


### q0

In [17]:
question = "what is the main treatment of type 1 diabetes mellitus?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

('assistant\n'
 '\n'
 'The main treatment of type 1 diabetes mellitus (T1DM) is insulin replacement '
 "therapy. Type 1 diabetes is an autoimmune disease in which the body's immune "
 'system attacks and destroys the cells in the pancreas that produce insulin, '
 'a hormone that regulates blood sugar levels. As a result, people with T1DM '
 'are unable to produce enough insulin and must rely on insulin replacement '
 'therapy to control their blood sugar levels.\n'
 '\n'
 'Insulin replacement therapy typically involves the use of insulin injections '
 'or an insulin pump. There are several types of insulin available, '
 'including:\n'
 '\n'
 '1. **Short-acting insulin**: This type of insulin is')


### q1

In [18]:
question = "How to cure asthma?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

('assistant\n'
 '\n'
 'Asthma is a chronic respiratory disease that cannot be cured, but it can be '
 'managed and controlled with proper treatment and lifestyle changes. The goal '
 'of asthma management is to control symptoms, prevent exacerbations, and '
 'improve quality of life.\n'
 '\n'
 '**Treatment Options:**\n'
 '\n'
 '1.  **Medications:** Asthma medications can help control symptoms and '
 'prevent exacerbations. Common medications include:\n'
 '    *   Inhaled corticosteroids (ICS) to reduce inflammation and swelling in '
 'the airways\n'
 '    *   Long-acting beta-agonists (LABAs) to open airways and improve '
 'breathing\n'
 '    *   Combination inhal')


### q2

In [19]:
question = "what is the indication for the drug alendronate sodium?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

('assistant\n'
 '\n'
 'Alendronate sodium is a bisphosphonate medication used to treat and prevent '
 'osteoporosis, a condition characterized by a decrease in bone density and an '
 "increased risk of fractures. It is also used to treat Paget's disease of "
 'bone, a condition that causes bones to become enlarged and deformed due to '
 'abnormal bone remodeling.\n'
 '\n'
 'Alendronate sodium works by inhibiting the activity of osteoclasts, which '
 'are cells that break down bone tissue. By reducing the activity of '
 'osteoclasts, alendronate sodium helps to increase bone density and reduce '
 'the risk of fractures.\n'
 '\n'
 'The indications for')


### q3

In [20]:
question = "what is the indication for the drug Methadone?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

('assistant\n'
 '\n'
 'Methadone is a synthetic opioid medication primarily used for the treatment '
 'of moderate to severe pain and for the management of opioid use disorder. '
 'The indications for methadone include:\n'
 '\n'
 '1. **Chronic Pain Management**: Methadone is used to treat moderate to '
 'severe chronic pain that is not adequately controlled by other pain '
 'medications. It is often used in patients with cancer, HIV/AIDS, or other '
 'chronic conditions that cause severe pain.\n'
 '2. **Opioid Use Disorder**: Methadone is used as a medication-assisted '
 'treatment (MAT) for opioid use disorder. It is used to help manage '
 'withdrawal symptoms and')


### q4

In [21]:
question = "what is the medication is used to treat hypothyroidism?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

('assistant\n'
 '\n'
 'Hypothyroidism is a condition in which the thyroid gland does not produce '
 'enough thyroid hormones. The medication used to treat hypothyroidism is '
 'called levothyroxine (T4).')


## JSL_MedS

* 'jsl_meds_q16_v1',
* 'jsl_meds_q4_v1',
* 'jsl_meds_q8_v1',

In [22]:
payload = json.dumps({
  "llm_name": "jsl_meds_q16_v1"
})
requests.request("POST", f"{APP_URL}/deploy", headers=HEADERS, data=payload).json()

{'message': 'deployment started'}

In [23]:
track_deployment_status(poll_every_seconds=60)

{'deployed': {'deployed_at': '2024-09-16 08:29:28 UTC',
              'device': 'auto',
              'max_context_length': 131072,
              'model': 'jsl_meds_q16_v1'},
 'system': {'gpus_info': [{'GPU Driver': '535.183.01',
                           'GPU ID': 0,
                           'GPU Load': '0.0 %',
                           'GPU Memory Free': '37769.0 MB',
                           'GPU Memory Total': '46068.0  MB',
                           'GPU Memory Used': '7826.0 MB',
                           'GPU Name': 'NVIDIA L40S',
                           'GPU Temperature': '38.0 °C'}],
            'is_cuda_available': True,
            'number_of_cuda_devices': 1,
            'torch_cpu_threads': 2}}


## q0

In [24]:
question = "what is the indication for the drug Methadone?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

('Methadone is indicated for the treatment of opioid dependence, chronic pain, '
 'and as part of palliative care for cancer patients.')


### q1

In [25]:
question = "what is the medication is used to treat hypothyroidism?"

prompt = f"""
### Question:
{question}
"""

response = generate(prompt = prompt)
print(response["message"])

'Levothyroxine is the primary medication used to treat hypothyroidism.'


### q2

In [26]:
prompt = f"""
Based on the following text, what age group is most susceptible to breast cancer?

## Text:
The exact cause of breast cancer is unknown. However, several risk factors can increase your likelihood of developing breast cancer, such as:
- A personal or family history of breast cancer
- A genetic mutation, such as BRCA1 or BRCA2
- Exposure to radiation
- Age (most commonly occurring in women over 50)
- Early onset of menstruation or late menopause
- Obesity
- Hormonal factors, such as taking hormone replacement therapy
"""

response = generate(prompt = prompt)
response["message"]

'The age group most susceptible to breast cancer is women over 50.'

### q3

In [27]:
prompt = f"""
A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.
Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

response = generate(prompt = prompt)
response["message"]

'E'